In [1]:
from IPython.core.display import HTML
with open('../style.css', 'r') as file:
    css = file.read()
HTML(css)

## Auxiliary Functions

We need the parser for first order formulas, hence we import it.

Formulas are represented as nested tuples.  In order to convert a string into a nested tuple we use the <tt>LogicParser</tt> that is found in the module <tt>folParser</tt>.  Our parser distinguishes variables and function symbol as follows:
- A word starting with a lower case letter is interpreted as a *variable*.
- A word starting with an upper case letter is assumed to be a *function* or *predicate symbol*.

In [2]:
import folParser as fp

In [3]:
def parse(s):
    return fp.LogicParser(s).parse()

The resolution calculus works with clauses.  The notebook `FOL-CNF.ipynb` implements the function $\texttt{normalize}(f)$ that turns a formula $f$ into a set of clauses.

In [4]:
%%capture
%run FOL-CNF.ipynb

The module `unify` implements [unification](https://en.wikipedia.org/wiki/Unification_(computer_science)) via the agorithm of [Martelli and Montanari](https://dl.acm.org/doi/pdf/10.1145/357162.357169).

In [5]:
%run Unification.ipynb

The function call $\texttt{arb}(S)$ returns an arbitrary element from the set $S$.

In [6]:
def arb(S):
    for x in S:
        return x

Given a literal $l$ the function $\texttt{complement}(l)$ computes the complement $\overline{\,l\,}$ of the literal $l$.

In [7]:
def complement(l):
    "Compute the complement of the literal l."
    match l:
        case ('¬', A):
            return A
        case A:
            return ('¬', A)

In [8]:
complement(('P', 'x'))

('¬', ('P', 'x'))

In [9]:
complement(('¬', ('P', 'x')))

('P', 'x')

Given a clause $C$, the function $\texttt{collectVariables}(C)$ computes the set of all variables occurring in $C$.  The function $\texttt{collectVariables}$ can also compute the variables in a literal or a term.

In [10]:
def collectVariables(C):
    if isinstance(C, frozenset):         # C is a clause
        return { x for literal in C 
                   for x in collectVariables(literal) 
               }
    if isinstance(C, str):               # C is a variable
        return { C }
    if C[0] == '¬':
        return collectVariables(C[1])    # C is a literal
    # C must be a term or atomic formula
    args = C[1:]
    return { x for t in args 
               for x in collectVariables(t) }

In [11]:
s = '∀g:∀c:(Grandparent(g, c) ↔ ∃p: (Parent(g, p) ∧ Parent(p, c)))'
f = parse(s)
f

('∀',
 'g',
 ('∀',
  'c',
  ('↔',
   ('Grandparent', 'g', 'c'),
   ('∃', 'p', ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c'))))))

In [12]:
Clauses = normalize(f)
Clauses

{frozenset({('Grandparent', 'g', 'c'),
            ('¬', ('Parent', 'a', 'c')),
            ('¬', ('Parent', 'g', 'a'))}),
 frozenset({('Parent', ('sk5', 'g', 'c'), 'c'),
            ('¬', ('Grandparent', 'g', 'c'))}),
 frozenset({('Parent', 'g', ('sk5', 'g', 'c')),
            ('¬', ('Grandparent', 'g', 'c'))})}

In [13]:
for C in Clauses:
    print(f'collectVariables({C}) = \n\t{collectVariables(C)}')

collectVariables(frozenset({('¬', ('Parent', 'g', 'a')), ('¬', ('Parent', 'a', 'c')), ('Grandparent', 'g', 'c')})) = 
	{'a', 'c', 'g'}
collectVariables(frozenset({('Parent', ('sk5', 'g', 'c'), 'c'), ('¬', ('Grandparent', 'g', 'c'))})) = 
	{'c', 'g'}
collectVariables(frozenset({('Parent', 'g', ('sk5', 'g', 'c')), ('¬', ('Grandparent', 'g', 'c'))})) = 
	{'c', 'g'}


In [14]:
from string import ascii_lowercase

The function $\texttt{renameVariables}(f, g)$ renames the variables in $f$ so that they are different from $g$.

In [15]:
def renameVariables(f, g):
    OldVars = collectVariables(f)
    NewVars = set(ascii_lowercase) - collectVariables(g)
    NewVars = sorted(list(NewVars))
    sigma   = { x: NewVars[i] for (i, x) in enumerate(OldVars) }
    return apply(f, sigma)

In [16]:
for C in Clauses:
    print(renameVariables(C, C))

frozenset({('Grandparent', 'e', 'd'), ('¬', ('Parent', 'e', 'b')), ('¬', ('Parent', 'b', 'd'))})
frozenset({('¬', ('Grandparent', 'b', 'a')), ('Parent', ('sk5', 'b', 'a'), 'a')})
frozenset({('¬', ('Grandparent', 'b', 'a')), ('Parent', 'b', ('sk5', 'b', 'a'))})


The function $\texttt{dictionaryToPairs}(D)$ converts a dictionary $D$ into a frozenset of key-value pairs.  This is needed because dictionaries are not hashable, while frozensets of pairs are hashable.

In [17]:
def 𝚍𝚒𝚌𝚝𝚒𝚘𝚗𝚊𝚛𝚢𝚃𝚘𝙿𝚊𝚒𝚛𝚜(D): 
    return frozenset({ (key, D[key]) for key in D })

In [18]:
dictionaryToPairs({'a': 1, 'b': 2})

frozenset({('a', 1), ('b', 2)})

# A Calculus for First Order Logic

The [resolution](https://en.wikipedia.org/wiki/Resolution_(logic)) rule is an inference rule that is defined as follows: If
 * $C_1$ and $C_2$ are clauses from first order logic,</li>
 * $p(s_1,\cdots,s_n)$ and $p(t_1,\cdots,t_n)$ are atomic formulas,</li> 
 * the syntactical equation $p(s_1,\cdots,s_n) \doteq p(t_1,\cdots,t_n)$ is solvable and
     $$ \mu = \mathtt{mgu}\bigl(p(s_1,\cdots,s_n), p(t_1,\cdots,t_n)\bigr), $$
then
$$\frac{C_1 \cup\{ p(s_1,\cdots,s_n)\} \quad\quad \{\neg p(t_1,\cdots,t_n)\} \cup C_2}{
                 C_1\mu \cup C_2\mu} 
$$
is an application of the resolution rule.

Given a two clauses <tt>C1</tt> and <tt>C2</tt>, the function $\texttt{resolve}(\texttt{C1}, \texttt{C2})$ computes a set of all clauses that can be inferred from <tt>C1</tt> and <tt>C2</tt> by applying the resolution rule.

In [19]:
def resolve(C1, C2):
    C2 = renameVariables(C2, C1)
    Result = set()
    for L1 in C1:
        for L2 in C2:
            mu = unify(L1, complement(L2))
            if mu != None:
                C1C2 = apply((C1 - { L1 }) | (C2 - { L2 }), mu)
                Result.add(C1C2)
    return Result

## Some Formulas for Testing

According to <a href="https://de.wikipedia.org/wiki/Uwe_Schöning">Uwe Schöning</a>, the theory of red dragons is
given by the following axioms:
<ol>
<li>
Every dragon is happy if all its children can fly:
$$ \forall x: \Bigl(\forall y: \big(\texttt{Child}(y,x) \rightarrow \texttt{CanFly}(y)\big) \rightarrow \texttt{Happy}(x)\Bigr) 
$$
</li>
<li> 
All red dragons can fly:
$$
 \forall x: \bigl(\texttt{Red}(x) \rightarrow \texttt{CanFly}(x)\bigr)
$$
</li>
<li> The children of red dragons are red themselves:
$$
\forall x: \bigl(\texttt{Red}(x) \rightarrow \forall y:\bigl( \texttt{Child}(y,x) \rightarrow \texttt{Red}(y)\bigr)\bigr)
$$
</li>
</ol>
We will show that these axioms imply that all red dragons are happy:
$$
 \forall x: \bigl(\texttt{Red}(x) \rightarrow \texttt{Happy}(x)\bigr)
$$
To this end, the formula stating that all red dragons can fly is negated.  Then we will show that the set consisting of the negated formula together with the axioms is inconsistent.  We start by defining the formulas.

In [20]:
s1 = '∀x:(∀y:(Child(y, x) → CanFly(y)) → Happy(x))'
s2 = '∀x:(Red(x) → CanFly(x))'
s3 = '∀x:(Red(x) → ∀y:(Child(y, x) → Red(y)))'
s4 = '¬∀x:(Red(x) → Happy(x))'

Next, the formulas are parsed and transformed into clauses.

In [21]:
f1 = parse(s1)
normalize(f1)

{frozenset({('Happy', 'x'), ('¬', ('CanFly', ('sk6', 'x')))}),
 frozenset({('Child', ('sk6', 'x'), 'x'), ('Happy', 'x')})}

In [22]:
f2 = parse(s2)
normalize(f2)

{frozenset({('CanFly', 'x'), ('¬', ('Red', 'x'))})}

In [23]:
f3 = parse(s3)
normalize(f3)

{frozenset({('Red', 'y'), ('¬', ('Child', 'y', 'x')), ('¬', ('Red', 'x'))})}

In [24]:
f4 = parse(s4)
normalize(f4)

{frozenset({('¬', ('Happy', ('sk7',)))}), frozenset({('Red', ('sk7',))})}

In [25]:
Clauses = normalize(f1) | normalize(f2) | normalize(f3) | normalize(f4)
Clauses

{frozenset({('Red', ('sk9',))}),
 frozenset({('¬', ('Happy', ('sk9',)))}),
 frozenset({('Red', 'y'), ('¬', ('Child', 'y', 'x')), ('¬', ('Red', 'x'))}),
 frozenset({('Happy', 'x'), ('¬', ('CanFly', ('sk8', 'x')))}),
 frozenset({('Child', ('sk8', 'x'), 'x'), ('Happy', 'x')}),
 frozenset({('CanFly', 'x'), ('¬', ('Red', 'x'))})}

We give names to the clauses in order to be able to refer to them.

In [26]:
C1 = frozenset({('Red', ('sk3',))})

In [27]:
C2 = frozenset({('¬', ('Happy', ('sk3',)))})

In [28]:
C3 = frozenset({('CanFly', 'x'), ('¬', ('Red', 'x'))})

In [29]:
C4 = frozenset({('Child', ('sk2', 'x'), 'x'), ('Happy', 'x')})

In [30]:
C5 = frozenset({('Happy', 'x'), ('¬', ('CanFly', ('sk2', 'x')))})

In [31]:
C6 = frozenset({('Red', 'y'), ('¬', ('Child', 'y', 'x')), ('¬', ('Red', 'x'))})

Now we are ready to show that the set consisting of these clauses is inconsistent.

In [32]:
C7 = arb(resolve(C1, C6))
C7

frozenset({('Red', 'a'), ('¬', ('Child', 'a', ('sk3',)))})

In [33]:
C8 = arb(resolve(C7, C4))
C8

frozenset({('Happy', ('sk3',)), ('Red', ('sk2', ('sk3',)))})

In [34]:
C9 = arb(resolve(C8, C2))
C9

frozenset({('Red', ('sk2', ('sk3',)))})

In [35]:
C10 = arb(resolve(C9, C3))
C10

frozenset({('CanFly', ('sk2', ('sk3',)))})

In [36]:
C11 = arb(resolve(C10, C5))
C11

frozenset({('Happy', ('sk3',))})

In [37]:
arb(resolve(C11, C2))

frozenset()

As we have derived the empty set, we have shown that all <b style="color:red;">communist</b> dragons are happy!

## Factorization

A calculus which only contains the resolution rule is not complete.  We also need the <em style="color:blue;">factorization</em> rule.  If
<ol>
<li> $C$ is a clause from first order logic,</li>
<li> $p(s_1,\cdots,s_n)$ and $p(t_1,\cdots,t_n)$ are atomic formulas,</li>
<li> the syntactical equation $p(s_1,\cdots,s_n)  \doteq p(t_1,\cdots,t_n)$ is solvable and 
     $$\mu = \mathtt{mgu}\bigl(p(s_1,\cdots,s_n), p(t_1,\cdots,t_n)\bigr), $$</li>
</ol>
then both 
$$ \frac{C \cup \bigl\{p(s_1,\cdots,s_n),\, p(t_1,\cdots,t_n)\bigl\}}{C\mu \cup \bigl\{p(s_1,\cdots,s_n)\mu\bigr\} } $$ 
and
$$ \frac{C \cup \bigl\{ \neg p(s_1,\cdots,s_n),\, \neg p(t_1,\cdots,t_n)\bigl\}}{C\mu \cup \bigl\{\neg p(s_1,\cdots,s_n)\mu\bigr\} }
$$ 
are applications of the factorization rule.

The function $\texttt{factorize}(C)$ takes a clause $C$ from first order logic and computes all clauses that can be derived from $C$ via factorization.

In [41]:
def factorize(C):
    Result = set()
    for L1 in C:
        for L2 in C:
            if L1 != L2:
                mu  = unify(L1, L2)
                if mu != None:
                    Cmu = apply(C, mu)
                    Result.add(Cmu)
    return Result

The clauses $C_1 := \forall x: \forall y: P(F(x),y) \vee \forall u: \forall v:P(u, G(v))$ and
$C_2 := \forall x: \forall y: \bigl(\neg P(F(x),y)\bigr) \vee \forall u: \forall v: \bigl(\neg P(u, G(v))\bigr)$
are inconsistent.  However, the resolution rule alone is not sufficient to show this.

In [38]:
C1 = arb(normalize(parse('∀x:∀y:P(F(x),y) ∨ ∀u:∀v:P(u,G(v))')))
C1

frozenset({('P', 'u', ('G', 'v')), ('P', ('F', 'x'), 'y')})

In [39]:
C2 = arb(normalize(parse('∀x:∀y:(¬P(F(x),y)) ∨ ∀u:∀v:(¬P(u,G(v)))')))
C2

frozenset({('¬', ('P', 'u', ('G', 'v'))), ('¬', ('P', ('F', 'x'), 'y'))})

In [42]:
C3 = arb(factorize(C1))
C3

frozenset({('P', ('F', 'x'), ('G', 'v'))})

In [43]:
C4 = arb(factorize(C2))
C4

frozenset({('¬', ('P', ('F', 'x'), ('G', 'v')))})

In [44]:
arb(resolve(C3, C4))

frozenset()

## Automatic Theorem Proving

Given two sets of clauses <tt>Cs1</tt> and <tt>Cs2</tt>, the function $\texttt{infere}(\texttt{Cs1}, \texttt{Cs2})$ returns all possible clauses that result from:
<ol>
    <li>the resolution of a clause $C_1 \in \texttt{Cs1}$ with a clause $C_2 \in \texttt{Cs2}$,</li>
    <li>the resolution of two clauses $C_1, C_2 \in \texttt{Cs1}$,</li>
    <li>the factorization of a clause $C \in \texttt{Cs1}$.</li>
</ol>

In [45]:
def infere(Clauses):
    Result =  { (C, (C1, C2)) for C1 in Clauses
                              for C2 in Clauses
                              for C  in resolve(C1, C2)
              }
    Result |= { (C, (C1,)) for C1 in Clauses for C in factorize(C1) }
    return Result 

In [46]:
def prettyPrint(Clauses):
    for C in Clauses:
        print(set(C))

In [47]:
prettyPrint(Clauses)

{('Red', ('sk9',))}
{('¬', ('Happy', ('sk9',)))}
{('¬', ('Red', 'x')), ('¬', ('Child', 'y', 'x')), ('Red', 'y')}
{('Happy', 'x'), ('¬', ('CanFly', ('sk8', 'x')))}
{('Child', ('sk8', 'x'), 'x'), ('Happy', 'x')}
{('CanFly', 'x'), ('¬', ('Red', 'x'))}


The function $\texttt{saturate}(\texttt{Cs})$ takes a set of clauses $\texttt{Cs}$ as input and tries to infere the empty clause.  If it is not possible to infer the empty clause, the function runs until memory is exhausted.

In [48]:
def saturate(Cs):
    Clauses = Cs.copy()
    cnt     = 1
    Reasons = {}
    while frozenset() not in Clauses:
        for (C, R) in infere(Clauses):
            if C not in Clauses:
                Reasons[C] = R
                Clauses.add(C)
        print(f'cnt = {cnt}, number of clauses: {len(Clauses)}')
        cnt += 1
    return Reasons

In [49]:
%%time
Reasons = saturate(Clauses)

cnt = 1, number of clauses: 19
cnt = 2, number of clauses: 125
cnt = 3, number of clauses: 7723
CPU times: user 1.09 s, sys: 10.5 ms, total: 1.1 s
Wall time: 1.1 s


Given a dictionary $\texttt{Reasons}$ and a clause $\texttt{clause}$, return a proof of $\texttt{clause}$.

In [50]:
def constructProof(clause, Reasons):
    if clause in Reasons:
        reason = Reasons[clause]
    else:
        return [f'Axiom:       {set(clause)}']
    if len(reason) == 1:
        (C,)  = reason
        Proof = constructProof(C, Reasons)
        Proof.append(f'Factorization: {set(C)} \n⊢' + ' ' * 12 + f'{set(clause)}')
    if len(reason) == 2:
        C1, C2  = reason
        ProofC1 = constructProof(C1, Reasons)
        ProofC2 = constructProof(C2, Reasons)
        Proof = update(ProofC1, ProofC2)
        Proof.append(f'Resolution:  {set(C1)},\n' + ' ' * 13 +
                     f'{set(C2)}  \n⊢' + ' ' * 12 + f'{set(clause)}')
    return Proof

In [51]:
def update(P1, P2):
    Result = P1
    for line in P2:
        if line not in Result:
            Result.append(line)
    return Result

In [52]:
Proof = constructProof(frozenset(), Reasons)
for line in Proof:
    print(line)

Axiom:       {('Red', ('sk9',))}
Axiom:       {('¬', ('Red', 'x')), ('¬', ('Child', 'y', 'x')), ('Red', 'y')}
Resolution:  {('Red', ('sk9',))},
             {('¬', ('Red', 'x')), ('¬', ('Child', 'y', 'x')), ('Red', 'y')}  
⊢            {('Red', 'a'), ('¬', ('Child', 'a', ('sk9',)))}
Axiom:       {('Child', ('sk8', 'x'), 'x'), ('Happy', 'x')}
Axiom:       {('¬', ('Happy', ('sk9',)))}
Resolution:  {('Child', ('sk8', 'x'), 'x'), ('Happy', 'x')},
             {('¬', ('Happy', ('sk9',)))}  
⊢            {('Child', ('sk8', ('sk9',)), ('sk9',))}
Resolution:  {('Red', 'a'), ('¬', ('Child', 'a', ('sk9',)))},
             {('Child', ('sk8', ('sk9',)), ('sk9',))}  
⊢            {('Red', ('sk8', ('sk9',)))}
Axiom:       {('Happy', 'x'), ('¬', ('CanFly', ('sk8', 'x')))}
Resolution:  {('Happy', 'x'), ('¬', ('CanFly', ('sk8', 'x')))},
             {('¬', ('Happy', ('sk9',)))}  
⊢            {('¬', ('CanFly', ('sk8', ('sk9',))))}
Axiom:       {('CanFly', 'x'), ('¬', ('Red', 'x'))}
Resolution:  {('¬', 